In [20]:
import requests
import json
import random
import time

# Base URL of the FastAPI server
BASE_URL = "http://localhost:8000"

# Optional: user_id for personalized /locations/recommended
# Get this from /auth/login if you want to test recommendations
USER_ID = "24f7bad2-ddff-4f8c-969d-2e63e22fd780"  # e.g. "6e0e5c7e-1234-4567-890a-bcdef1234567"

HEADERS = {"X-User-ID": USER_ID} if USER_ID else {}

print("BASE_URL =", BASE_URL)
print("USER_ID =", USER_ID)


BASE_URL = http://localhost:8000
USER_ID = 24f7bad2-ddff-4f8c-969d-2e63e22fd780


In [21]:
location_names = [
    "Victoria Peak", "Lion Rock", "Dragon's Back", "Mount Parker", "Lantau Peak",
    "Tai Mo Shan", "Sharp Peak", "Sunset Peak", "Needle Hill", "Ma On Shan",
    "High West", "Pat Sin Leng", "Jardine's Lookout", "The Twins", "Castle Peak",
    "Pineapple Mountain", "Shing Mun Reservoir", "Aberdeen Peak", "Clear Water Bay Peak",
    "Ap Lei Chau Trail", "Cape D’Aguilar", "Sai Wan Peak", "Hawk's Nest", "Tung Chung Valley"
]

areas = ["Central", "Kowloon", "New Territories", "Lantau", "Hong Kong Island"]
price_levels = [1, 2, 3, 4]

bulk_created_ids = []

for name in location_names:
    payload = {
        "name": name,
        "description": f"A scenic location known as {name}.",
        "maps_url": "https://maps.example.com",
        "price_level": random.choice(price_levels),
        "area": random.choice(areas),
    }

    url = f"{BASE_URL}/locations/"
    try:
        res = requests.post(url, json=payload)
        print(f"Created {name} → Status {res.status_code}")
        print("Status:", res.status_code)
        print("Response:", res.text)
        data = res.json()
        loc_id = data.get("id")
        if loc_id:
            bulk_created_ids.append(loc_id)
    except Exception as e:
        print(f"Error creating {name}: {e}")

time.sleep(0.5)
print("\nTotal created:", len(bulk_created_ids))
bulk_created_ids[:5]


Created Victoria Peak → Status 200
Status: 200
Response: {"name":"Victoria Peak","description":"A scenic location known as Victoria Peak.","maps_url":"https://maps.example.com","price_level":1,"area":"Kowloon","region":null,"summary":null,"duration":null,"opening_hours":null,"id":"1fb76748-b06b-469c-b961-dd07b59d34ed","created_at":"2025-11-23T18:17:17.298596+08:00"}
Created Lion Rock → Status 200
Status: 200
Response: {"name":"Lion Rock","description":"A scenic location known as Lion Rock.","maps_url":"https://maps.example.com","price_level":1,"area":"Central","region":null,"summary":null,"duration":null,"opening_hours":null,"id":"04eec270-9e18-444f-9f75-4164ea3dc37d","created_at":"2025-11-23T18:17:17.370909+08:00"}
Created Dragon's Back → Status 200
Status: 200
Response: {"name":"Dragon's Back","description":"A scenic location known as Dragon's Back.","maps_url":"https://maps.example.com","price_level":3,"area":"Kowloon","region":null,"summary":null,"duration":null,"opening_hours":nul

['1fb76748-b06b-469c-b961-dd07b59d34ed',
 '04eec270-9e18-444f-9f75-4164ea3dc37d',
 '24b4e70e-3078-4a48-a265-823dab5f7d30',
 '5611b08e-45a4-45b0-9199-6f47bac81971',
 'd1c75104-dd42-4da7-b741-9c4d858d78b8']

In [22]:
url = f"{BASE_URL}/locations/"
res = requests.get(url)

print("Status:", res.status_code)
all_locations = res.json()
print("Total locations returned:", len(all_locations))
print(json.dumps(all_locations[:3], indent=2))  # show first 3


Status: 200
Total locations returned: 24
[
  {
    "name": "Victoria Peak",
    "description": "A scenic location known as Victoria Peak.",
    "maps_url": "https://maps.example.com",
    "price_level": 1,
    "area": "Kowloon",
    "region": null,
    "summary": null,
    "duration": null,
    "opening_hours": null,
    "id": "1fb76748-b06b-469c-b961-dd07b59d34ed",
    "created_at": "2025-11-23T18:17:17.298596+08:00"
  },
  {
    "name": "Lion Rock",
    "description": "A scenic location known as Lion Rock.",
    "maps_url": "https://maps.example.com",
    "price_level": 1,
    "area": "Central",
    "region": null,
    "summary": null,
    "duration": null,
    "opening_hours": null,
    "id": "04eec270-9e18-444f-9f75-4164ea3dc37d",
    "created_at": "2025-11-23T18:17:17.370909+08:00"
  },
  {
    "name": "Dragon's Back",
    "description": "A scenic location known as Dragon's Back.",
    "maps_url": "https://maps.example.com",
    "price_level": 3,
    "area": "Kowloon",
    "region

In [23]:
# Example: filter by area and price_level
url = f"{BASE_URL}/locations/?area=Central&price_level=2"
res = requests.get(url)

print("Status:", res.status_code)
filtered_locations = res.json()
print("Filtered locations count:", len(filtered_locations))
print(json.dumps(filtered_locations[:3], indent=2))


Status: 200
Filtered locations count: 1
[
  {
    "name": "Cape D\u2019Aguilar",
    "description": "A scenic location known as Cape D\u2019Aguilar.",
    "maps_url": "https://maps.example.com",
    "price_level": 2,
    "area": "Central",
    "region": null,
    "summary": null,
    "duration": null,
    "opening_hours": null,
    "id": "44210137-da69-4b06-a04b-86f7167aa6ab",
    "created_at": "2025-11-23T18:17:17.731272+08:00"
  }
]


In [24]:
if bulk_created_ids:
    test_location_id = bulk_created_ids[0]
    print("Using test_location_id from bulk_created_ids:", test_location_id)
elif all_locations:
    test_location_id = all_locations[0]["id"]
    print("No bulk_created_ids, using first from all_locations:", test_location_id)
else:
    test_location_id = None
    print("No locations available to test.")

test_location_id


Using test_location_id from bulk_created_ids: 1fb76748-b06b-469c-b961-dd07b59d34ed


'1fb76748-b06b-469c-b961-dd07b59d34ed'

In [25]:
if not test_location_id:
    raise RuntimeError("No test_location_id available. Run previous cells first.")

url = f"{BASE_URL}/locations/{test_location_id}"
res = requests.get(url)

print("Status:", res.status_code)
print(json.dumps(res.json(), indent=2))


Status: 200
{
  "location": {
    "name": "Victoria Peak",
    "description": "A scenic location known as Victoria Peak.",
    "maps_url": "https://maps.example.com",
    "price_level": 1,
    "area": "Kowloon",
    "region": null,
    "summary": null,
    "duration": null,
    "opening_hours": null,
    "id": "1fb76748-b06b-469c-b961-dd07b59d34ed",
    "created_at": "2025-11-23T18:17:17.298596+08:00"
  },
  "images": [],
  "tags": []
}


In [26]:
update_payload = {
    "name": "Updated Test Location",
    "price_level": 3,
}

url = f"{BASE_URL}/locations/{test_location_id}"
res = requests.put(url, json=update_payload)

print("Status:", res.status_code)
print(json.dumps(res.json(), indent=2))


Status: 200
{
  "name": "Updated Test Location",
  "description": "A scenic location known as Victoria Peak.",
  "maps_url": "https://maps.example.com",
  "price_level": 3,
  "area": "Kowloon",
  "region": null,
  "summary": null,
  "duration": null,
  "opening_hours": null,
  "id": "1fb76748-b06b-469c-b961-dd07b59d34ed",
  "created_at": "2025-11-23T18:17:17.298596+08:00"
}


In [27]:
import os

image_path = "test_image.jpg"  # change if needed

if os.path.exists(image_path):
    url = f"{BASE_URL}/locations/{test_location_id}/images"
    with open(image_path, "rb") as f:
        files = {"file": (os.path.basename(image_path), f, "image/jpeg")}
        res = requests.post(url, files=files)

    print("Status:", res.status_code)
    print(json.dumps(res.json(), indent=2))
else:
    print(f"Image file not found: {image_path}. Skipping image upload test.")


Image file not found: test_image.jpg. Skipping image upload test.


In [28]:
tags_payload = {
    "tags": ["hiking", "scenic", "sunset"]
}

url = f"{BASE_URL}/locations/{test_location_id}/tags"
res = requests.post(url, json=tags_payload)

print("Status:", res.status_code)
tags_response = res.json()
print(json.dumps(tags_response, indent=2))

tag_ids = [t.get("id") for t in tags_response.get("added_tags", []) if t.get("id") is not None]
tag_ids


Status: 200
{
  "added_tags": [
    {
      "name": "hiking",
      "id": 1
    },
    {
      "name": "scenic",
      "id": 2
    },
    {
      "name": "sunset",
      "id": 3
    }
  ]
}


[1, 2, 3]

In [29]:
if tag_ids:
    tag_id = tag_ids[0]
    url = f"{BASE_URL}/locations/{test_location_id}/tags/{tag_id}"
    res = requests.delete(url)

    print("Status:", res.status_code)
    print(json.dumps(res.json(), indent=2))
else:
    print("No tag_ids available to delete.")


Status: 200
{
  "message": "Tag removed"
}


In [30]:
if not USER_ID:
    print("USER_ID is not set. Skipping /locations/recommended test.")
else:
    url = f"{BASE_URL}/locations/recommended?limit=10"
    res = requests.get(url, headers=HEADERS)
    print("Status:", res.status_code)
    print(json.dumps(res.json(), indent=2))


Status: 200
[
  {
    "location": {
      "name": "Sunset Peak",
      "description": "A scenic location known as Sunset Peak.",
      "maps_url": "https://maps.example.com",
      "price_level": 4,
      "area": "Lantau",
      "region": null,
      "summary": null,
      "duration": null,
      "opening_hours": null,
      "id": "a151d28a-53f9-4c99-b73a-0088482dd862",
      "created_at": "2025-11-23T18:17:17.496344+08:00"
    },
    "images": [],
    "tags": []
  },
  {
    "location": {
      "name": "Tung Chung Valley",
      "description": "A scenic location known as Tung Chung Valley.",
      "maps_url": "https://maps.example.com",
      "price_level": 3,
      "area": "Central",
      "region": null,
      "summary": null,
      "duration": null,
      "opening_hours": null,
      "id": "f383b465-9f7e-4077-9093-7039bfdeb3c1",
      "created_at": "2025-11-23T18:17:17.821040+08:00"
    },
    "images": [],
    "tags": []
  },
  {
    "location": {
      "name": "Clear Water Bay Pe

In [31]:
cleanup = False  # set to True if you want to delete the test location

if cleanup and test_location_id:
    url = f"{BASE_URL}/locations/{test_location_id}"
    res = requests.delete(url)
    print("Status:", res.status_code)
    print(json.dumps(res.json(), indent=2))
elif not cleanup:
    print("Cleanup is disabled. Set cleanup = True to delete the test location.")
else:
    print("No test_location_id to delete.")


Cleanup is disabled. Set cleanup = True to delete the test location.
